In [ ]:
from ema_workbench import Scenario
from ema_workbench.analysis import prim
from problem_formulation import get_model_for_problem_formulation
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench import load_results
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
dike_model, planning_steps = get_model_for_problem_formulation("Gorssel")

# Scenario selection

In [ ]:
ACTOR = "genscen_Gorssel_50000_08-06-2021-01-09-07.tar.gz"
experiments, outcomes = load_results("data/generated/" + ACTOR)

In [ ]:
selected = pd.read_csv("data/gorssel_scenarios.csv")
selected
#Do something

In [ ]:
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

# Optimisation

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

def optimize(scenario, nfe, model, converge_metrics, epsilons):


    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence


results = []
for scenario in scenarios:
    convergence_metrics = [HyperVolume(minimum=[0,0,0,0], maximum=[3, 2,1.01,1.01]),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(model.outcomes)
    
    results.append(optimize(scenario, 1e4, dike_model, convergence_metrics, epsilons))


# Regret

In [ ]:
thresholds = {}

# Satisficing

In [ ]:
thresholds = {}